In [1]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import melt
import gezi

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
class CommonPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(CommonPipeline, self).__init__(batch_size, num_threads, device_id)
        pass

    def base_define_graph(self, inputs, labels):
        pass

In [4]:
tfrecord =  ['/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v1/tfrecords/2020010821/tfrecord.88655.36416']

In [5]:
tfrecord_idx = ['/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v1/tfrecords.idx/2020010821/tfrecord.88655.36416']

In [6]:
import nvidia.dali.tfrecord as tfrec

class TFRecordPipeline(CommonPipeline):
    def __init__(self, batch_size, num_threads, device_id, num_gpus=1):
        super(TFRecordPipeline, self).__init__(batch_size, num_threads, device_id)
        self.input = ops.TFRecordReader(path=tfrecord,
                                        index_path=tfrecord_idx,
                                        features={"click" : tfrec.FixedLenFeature(([1]), tfrec.int64, 0),
                                                  "value" : tfrec.FixedLenFeature(([1]), tfrec.int64, 0),
                                                  "index": tfrec.VarLenFeature(tfrec.int64,  0),
                                                  "field": tfrec.VarLenFeature(tfrec.int64,  0),
                                                  "id": tfrec.FixedLenFeature((), tfrec.string, ''),
                                                  "duration": tfrec.VarLenFeature(tfrec.int64,  0)},
                                        #random_shuffle=True
                                        num_shards=num_gpus,
                                        shard_id=device_id,
                                       )

    def define_graph(self):
        inputs = self.input(name="Reader")
        return inputs['field'], inputs['index'], inputs['field']

In [7]:
BATCH_SIZE = 512
DEVICES = 1
deviece_id = 0
N = 8
N = 1
pipe_name = TFRecordPipeline
pipes = [TFRecordPipeline(batch_size=BATCH_SIZE, num_threads=2, device_id = device_id, num_gpus = DEVICES) for device_id in range(DEVICES)]

In [8]:
import tensorflow as tf
import nvidia.dali.plugin.tf as dali_tf
import time

ModuleNotFoundError: No module named 'nvidia.dali_tf_plugin'

In [ ]:
batch_size = 16
pipe = TFRecordPipeline(batch_size=batch_size, num_threads=4, device_id=0)
pipe.build()

In [ ]:
pipes = [TFRecordPipeline(batch_size=BATCH_SIZE, num_threads=2, device_id = device_id, num_gpus = DEVICES) for device_id in range(DEVICES)]
daliop = dali_tf.DALIIterator()

ids, fields, indexes = [], [], []
for d in range(DEVICES):
  with tf.device('/cpu'):
    field, index, id = daliop(pipeline = pipes[d],
      shapes = [(BATCH_SIZE), ()],
      dtypes = [tf.int64, tf.int64, tf.string], sparse = [True, True, False])
    index = melt.sparse_tensor_to_dense(index)
    field = melt.sparse_tensor_to_dense(field)
    indexes.append(index)
    fields.append(field)
    ids.append(id)
    if len(indexes) == 10:
      break

In [ ]:
sess = tf.Session()
x, y = sess.run([indexes, fields])

In [ ]:
y[0].shape

In [ ]:
x[0].shape

In [ ]:
36416 / 512

In [ ]:
for i in range(100):
  x, y, z = sess.run([indexes, vals, fields])
  print(i, x[0].shape)